# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [23]:
# Import libraries
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn as skl
import matplotlib.pyplot as plt


In [24]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [25]:
student_data[student_data.passed=='yes'].shape[0]

265

In [26]:
student_data.dtypes

school        object
sex           object
age            int64
address       object
famsize       object
Pstatus       object
Medu           int64
Fedu           int64
Mjob          object
Fjob          object
reason        object
guardian      object
traveltime     int64
studytime      int64
failures       int64
schoolsup     object
famsup        object
paid          object
activities    object
nursery       object
higher        object
internet      object
romantic      object
famrel         int64
freetime       int64
goout          int64
Dalc           int64
Walc           int64
health         int64
absences       int64
passed        object
dtype: object

In [27]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1]-1
n_passed = student_data[student_data.passed=='yes'].shape[0]
n_failed = student_data[student_data.passed=='no'].shape[0]
grad_rate = 100 * n_passed / (n_passed + n_failed)
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.00%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [28]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [29]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

preproc_sd = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(preproc_sd.columns), list(preproc_sd.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [30]:
# First, decide how many training vs test samples you want
num_all = preproc_sd.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train
shuffled_preproc_sd = preproc_sd.reindex(np.random.permutation(preproc_sd.index))
# Change indices on the labels to match the shuffling.
shuffled_indices = shuffled_preproc_sd.index.values
shuffled_labels = y_all.reindex(shuffled_indices)

In [31]:
# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
X_train = shuffled_preproc_sd.head(num_train).values
y_train = shuffled_labels.head(num_train).values
X_test = shuffled_preproc_sd.tail(num_test).values
y_test = shuffled_labels.tail(num_test).values
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


I think there are a couple features that might be the most important based on my experience with teaching. First, attendance is key! Another good feature to examine would be the school and family support they receive.

In [32]:
# passed_data = shuffled_labels.values
# passed_data[passed_data == 'yes'] = 1
# passed_data[passed_data == 'no'] = 0
# absence_data = shuffled_preproc_sd['absences'].values
# famsupport_data = shuffled_preproc_sd['famsup'].values

In [33]:
# plt.scatter(absence_data,passed_data)

In [34]:
# plt.scatter(famsupport_data,passed_data)

Ok, does not reveal much. But, I did see some good python moves for modifying arrays using logical matching.

## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [35]:
# Taining data partitioning
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]
X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

In [36]:
# Load up the GridSearch
from sklearn.grid_search import GridSearchCV

### Decision Tree
Intuitively, this data appears to match the "20 questions" approach mentioned by the course materials.



In [37]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn import tree
dtc = tree.DecisionTreeClassifier(criterion="entropy")

Try out grid search with the following parameter selections:

1. criterion = "gini", "entropy"
2. min_samples_split = 2, 4, 8, 16
3. max_features = None, sqrt, log2

In [49]:
dtc_params = {'criterion':("gini","entropy"), 'min_samples_split':(2,4,8,16), 'max_features':("auto","sqrt","log2"),
             'max_depth':np.arange(1,31,1)}
f1scorer = skl.metrics.make_scorer( lambda yt, yp : skl.metrics.f1_score(yt, yp, pos_label='yes') ) 
tuned_dtc = GridSearchCV(dtc, dtc_params, f1scorer)
tuned_dtc.fit(X_train, y_train)
tuned_dtc.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features='log2', max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=4, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [50]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target, y_pred, pos_label='yes')

#### Evaluation DT Classifier with Varying Sized Training Data
From the data below, when I use entropy for splits, I get a lower F1 score with lower data (makes sense). The score increases with more data, which should happen as more data is added.

In [51]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
#     print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

print "Non-tuned DT Classifier"
train_predict(dtc, X_train_100, y_train_100, X_test, y_test)
train_predict(dtc, X_train_200, y_train_200, X_test, y_test)
train_predict(dtc, X_train, y_train, X_test, y_test)
print "GridSearch DT Classifier"
train_predict(tuned_dtc, X_train_100, y_train_100, X_test, y_test)
train_predict(tuned_dtc, X_train_200, y_train_200, X_test, y_test)
train_predict(tuned_dtc, X_train, y_train, X_test, y_test)

Non-tuned DT Classifier
------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.748091603053
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.736842105263
------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.785185185185
GridSearch DT Classifier
------------------------------------------
Training set size: 100
Training GridSearchCV...
Done!
Training time (secs): 5.006
Predicting labels using GridSearchCV...
Done!
Pre

### Neural Network Model
Neural network models are mentioned as good for dealing with lots of features. The sklearn MLP classifier documentation states that it can handle non-linear data. Also listed in the documentation is a recommendation to scale the data. See example: 

In [41]:
# from sklearn.preprocessing import StandardScaler
# # test with the size 100 data set
# scaler100 = StandardScaler()
# scaler100.fit(X_train_100)
# X_train_100_s = scaler100.transform(X_train_100)
# X_test_s = scaler100.transform(X_test)

With scaled training and test data in hand, time to play with the MLP classifier. Edit: looks like I cannot use it! Not in the sklearn library. :(

### Bayesian Model
The sklearn documentation looks good: Naive Bayes is fast. Time to try it out!

In [42]:
from sklearn.naive_bayes import GaussianNB
# GaussianNB can accept sigma and theta as parameters, but I will try it empty.
gnb = GaussianNB()
train_predict(gnb, X_train_100, y_train_100, X_test, y_test)
train_predict(gnb, X_train_200, y_train_200, X_test, y_test)
train_predict(gnb, X_train, y_train, X_test, y_test)

------------------------------------------
Training set size: 100
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.769230769231
------------------------------------------
Training set size: 200
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.779411764706
------------------------------------------
Training set size: 300
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.80303030303


With minimal tuning, it works pretty well. The F1 scores are close to those of the DT above.

### Ensembles
My last step will evaluate and choose an ensemble method. The data set has lots of features that take different values. Much like the email examples in the lectures, this project could benefit from an ensemble of simpler classifiers. I experimented with two classifiers so far: DT and Gaussian NB, so I will see how they perform in a simple bagging classifier and then in a voting classifier.

In [60]:
# Bagger!
from sklearn.ensemble import BaggingClassifier
# I selected to have smaller sample and feature sets, but more estimators. The DT will use the entropy criterion.
baggingClf_DT = BaggingClassifier(tree.DecisionTreeClassifier(criterion="entropy"), max_samples=0.3, max_features=0.3, n_estimators=20)
train_predict(baggingClf_DT, X_train_100, y_train_100, X_test, y_test)
train_predict(baggingClf_DT, X_train_200, y_train_200, X_test, y_test)
train_predict(baggingClf_DT, X_train, y_train, X_test, y_test)

------------------------------------------
Training set size: 100
Training BaggingClassifier...
Done!
Training time (secs): 0.038
Predicting labels using BaggingClassifier...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.787096774194
------------------------------------------
Training set size: 200
Training BaggingClassifier...
Done!
Training time (secs): 0.035
Predicting labels using BaggingClassifier...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.765100671141
------------------------------------------
Training set size: 300
Training BaggingClassifier...
Done!
Training time (secs): 0.036
Predicting labels using BaggingClassifier...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.818791946309


In [70]:
# Next, try out the same bagging structure but with Gaussian NB.
baggingClf_GNB = BaggingClassifier(GaussianNB(), max_samples=0.3, max_features=0.3, n_estimators=20)
train_predict(baggingClf_GNB, X_train_100, y_train_100, X_test, y_test)
train_predict(baggingClf_GNB, X_train_200, y_train_200, X_test, y_test)
train_predict(baggingClf_GNB, X_train, y_train, X_test, y_test)

------------------------------------------
Training set size: 100
Training BaggingClassifier...
Done!
Training time (secs): 0.015
Predicting labels using BaggingClassifier...
Done!
Prediction time (secs): 0.004
F1 score for test set: 0.780141843972
------------------------------------------
Training set size: 200
Training BaggingClassifier...
Done!
Training time (secs): 0.015
Predicting labels using BaggingClassifier...
Done!
Prediction time (secs): 0.004
F1 score for test set: 0.783216783217
------------------------------------------
Training set size: 300
Training BaggingClassifier...
Done!
Training time (secs): 0.014
Predicting labels using BaggingClassifier...
Done!
Prediction time (secs): 0.004
F1 score for test set: 0.797202797203


The performance looks similar to both single DT and single Gaussian NB classifiers. Next, explore the voting classifier with a DT and a GNB. I will use the tuned DTC and GNB from the prior code.

In [73]:
from sklearn.ensemble import VotingClassifier
votingClf = VotingClassifier(estimators=[('tuned_dtc', tuned_dtc), ('gnb', gnb), ('dtc', dtc)], voting='hard')
#train_predict(votingClf, X_train_100, y_train_100, X_test, y_test)
votingClf.fit(X_train, y_train)

ValueError: pos_label='yes' is not a valid label: array([0, 1])

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [43]:
# TODO: Fine-tune your model and report the best F1 score